In [ ]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import os
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report,confusion_matrix
from warnings import filterwarnings
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        os.path.join(dirname, filename)

Data Preperation
To prepare our data we'll have to first create our labels and import our images, then split it into training and testing data for the model

In [ ]:
Data Preperation
To prepare our data we'll have to first create our labels and import our images, then split it into training and testing data for the model

In [ ]:
# append all images to lists
X_train = []
y_train = []
image_size = 150
for i in labels:
    folderPath = os.path.join('../input/brain-tumor-classification-mri','Training',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size, image_size))
        X_train.append(img)
        y_train.append(i)
        
for i in labels:
    folderPath = os.path.join('../input/brain-tumor-classification-mri','Testing',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = cv2.resize(img,(image_size,image_size))
        X_train.append(img)
        y_train.append(i)

In [ ]:
# convert to arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

In [ ]:
# create figure to look at sample image
k=0
fig, ax = plt.subplots(1,4,figsize=(20,20))
fig.text(s='Example Image From Each Label',
         size=18,
         fontweight='bold',
         fontname='monospace',
         color='black',
         y=0.62,
         x=0.4,
         alpha=0.8)

# upload random image
import random
for i in labels:
    j=random.randint(0, len(X_train)-1)
    while True :
        if y_train[j]==i:
            ax[k].imshow(X_train[j])
            ax[k].set_title(y_train[j])
            ax[k].axis('off')
            k+=1
            break
        j+=1

In [ ]:
# shuffle data
X_train, y_train = shuffle(X_train,y_train, random_state=101)

In [ ]:
# look at shape
X_train.shape

In [ ]:
# create the training and testing split from our data
X_train,X_test,y_train,y_test = train_test_split(X_train,y_train,test_size=0.1,random_state=101)

In [ ]:

# convert labels to categorical numbers
y_train_new = []
for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = tf.keras.utils.to_categorical(y_train)


y_test_new = []
for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = tf.keras.utils.to_categorical(y_test)

Transfer Learning
To approach this image classifcation problem we'll use transfer learning, applying the weights on a previously successful model to our current one. The one I've selected is EfficientNetB0 as it isn't as computationally demanding and is well trained on ImageNet.

In [ ]:
# import EfficientNetB0 weights
effnet = EfficientNetB0(weights='imagenet',
                        include_top=False,
                        input_shape=(image_size,image_size,3))


In [ ]:
# apply weights to our model
model = effnet.output

# global average for easier computation
model = tf.keras.layers.GlobalAveragePooling2D()(model)

# dropout to avoid overfitting
model = tf.keras.layers.Dropout(rate=0.5)(model)

# output layer
model = tf.keras.layers.Dense(4,activation='softmax')(model)
model = tf.keras.models.Model(inputs=effnet.input, outputs = model)

# look at model summary
model.summary()

In [ ]:
Model: "functional_1"


In [ ]:
# compile the model
model.compile(loss='categorical_crossentropy',
              optimizer = 'Adam',
              metrics= ['accuracy'])

In [ ]:

# get callbacks to keep up with model
tensorboard = TensorBoard(log_dir = 'logs')
checkpoint = ModelCheckpoint("effnet.keras",monitor="val_accuracy",save_best_only=True,mode="auto",verbose=1)
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', factor = 0.3, patience = 2, min_delta = 0.001,
                              mode='auto',verbose=1)

In [ ]:
# train the model on our data
history = model.fit(X_train,y_train,validation_split=0.1, epochs =12, verbose=1, batch_size=32,
                   callbacks=[tensorboard,checkpoint,reduce_lr])


Model Evaluation
To evaluate our model's performance we'll look at how its accuracy and loss change over time, as well as its confusion matrix to ensure it's making the right classification.

In [ ]:
# look at chart of accuracy & loss
filterwarnings('ignore')

epochs = [i for i in range(12)]
fig, ax = plt.subplots(1,2,figsize=(14,7))
train_acc = history.history['accuracy']
train_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

fig.text(0.5, 0.9, 'Performance Over Epochs',
         ha='center',
         fontsize=20,
         fontweight='bold')

sns.despine()
ax[0].plot(epochs, train_acc, marker='o',color='blue',
           label = 'Training')
ax[0].plot(epochs, val_acc, marker='o',color='red',
           label = 'Validation')
ax[0].legend(frameon=False)
ax[0].set_xlabel('Epochs')
ax[0].set_ylabel('Accuracy')

sns.despine()
ax[1].plot(epochs, train_loss, marker='o',color='blue',
           label ='Training')
ax[1].plot(epochs, val_loss, marker='o',color='red',
           label = 'Validation')
ax[1].legend(frameon=False)
ax[1].set_xlabel('Epochs')
ax[1].set_ylabel('Loss')

fig.show()

In [ ]:
# create predicted values to compare vs actual ones
pred = model.predict(X_test)
pred = np.argmax(pred,axis=1)
y_test_new = np.argmax(y_test,axis=1)


In [ ]:
# plot confusion matrix
fig,ax=plt.subplots(1,1,figsize=(14,7))
sns.heatmap(confusion_matrix(y_test_new,pred),
            ax=ax,xticklabels=labels,
            yticklabels=labels,
            annot=True,
            cmap='Blues',
            alpha=0.7,
            linewidths=2,
            linecolor='black',
            cbar=False)
fig.text(s='Heatmap of the Confusion Matrix',size=18,fontweight='bold',
             fontname='monospace',color='black',y=0.92,x=0.28,alpha=0.8)

plt.show()
